In [1]:
#Imports
import pandas as pd
import ast

In [ ]:
#Global Variables
DATA_PATH = '../../data/raw/'
MOVIES_FILE = DATA_PATH + "movies_metadata.csv"
ISO_639_FILE = DATA_PATH + "language-codes.csv"
ISO_3166_FILE = DATA_PATH + "country-codes.csv"

CLEAN_DATA_PATH = '../../data/clean/'
CLEAN_MOVIES_FILE = CLEAN_DATA_PATH + "clean_movies_metadata.csv"
COLLECTIONS_FILE = CLEAN_DATA_PATH + "movie_collections.csv"
GENRE_FILE = CLEAN_DATA_PATH + "movie_genres.csv"
PRODUCTION_COMPANIES_FILE = CLEAN_DATA_PATH + "movie_production_companies.csv"
PRODUCTION_COUNTRIES_FILE = CLEAN_DATA_PATH + "movie_production_countries.csv"
SPOKEN_LANGUAGES_FILE = CLEAN_DATA_PATH + "movie_spoken_languages.csv"

In [3]:
#ISO639-1 dictionary
iso_639_pd = pd.read_csv(ISO_639_FILE, header=0)
iso_639_dict = {row['alpha2']: row['English'] for index, row in iso_639_pd.iterrows()}

#ISO3166-1 dictionary
iso_3166_pd = pd.read_csv(ISO_3166_FILE, header=0)
iso_3166_dict = {row['Code']: row['Name'] for index, row in iso_3166_pd.iterrows()}

## CLEANING AND PREPERAING MOVIES DATASET

In [4]:
#First look at raw data
movies_pd = pd.read_csv(MOVIES_FILE, low_memory=False)
movies_pd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
#df info
movies_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Since data will be used as context in RAG, null values are acteptable in some cases.
Non-null variables are: id, title and overview

In [6]:
#Remove unneccessary columns 
columns_to_remove = ['adult','homepage', 'poster_path', 'status', 'video', 'tagline']
movies_pd = movies_pd.drop(columns=columns_to_remove)

In [7]:
#df info
movies_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   imdb_id                45449 non-null  object 
 5   original_language      45455 non-null  object 
 6   original_title         45466 non-null  object 
 7   overview               44512 non-null  object 
 8   popularity             45461 non-null  object 
 9   production_companies   45463 non-null  object 
 10  production_countries   45463 non-null  object 
 11  release_date           45379 non-null  object 
 12  revenue                45460 non-null  float64
 13  runtime                45203 non-null  float64
 14  spoken_languages       45460 non-null  object 
 15  ti

In [8]:
#Droping null values in id, title and overview
movies_pd = movies_pd.dropna(subset=['id', 'title', 'overview'])

#Since title and overview are strings, empty strings are considered null values
movies_pd = movies_pd[movies_pd['overview'].str.strip().astype(bool)]
movies_pd = movies_pd[movies_pd['title'].str.strip().astype(bool)]

In [9]:
#df info
movies_pd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44501 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4431 non-null   object 
 1   budget                 44501 non-null  object 
 2   genres                 44501 non-null  object 
 3   id                     44501 non-null  object 
 4   imdb_id                44486 non-null  object 
 5   original_language      44491 non-null  object 
 6   original_title         44501 non-null  object 
 7   overview               44501 non-null  object 
 8   popularity             44501 non-null  object 
 9   production_companies   44501 non-null  object 
 10  production_countries   44501 non-null  object 
 11  release_date           44430 non-null  object 
 12  revenue                44501 non-null  float64
 13  runtime                44501 non-null  float64
 14  spoken_languages       44501 non-null  object 
 15  title  

Changing datatypes

In [10]:
#Objects to int or float
types_to_convert = ['budget', 'id', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']
for columns in types_to_convert:
    movies_pd[columns] = pd.to_numeric(movies_pd[columns], errors='coerce')


In [11]:
#Changing release_date to datetime
movies_pd['release_date'] = pd.to_datetime(movies_pd['release_date'], errors='coerce')

In [12]:
#Translating original_language from ISO 639-1 to English
movies_pd['original_language'] = movies_pd['original_language'].apply(lambda x: iso_639_dict.get(x, None))

In [13]:
#df info
movies_pd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44501 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4431 non-null   object        
 1   budget                 44501 non-null  int64         
 2   genres                 44501 non-null  object        
 3   id                     44501 non-null  int64         
 4   imdb_id                44486 non-null  object        
 5   original_language      44140 non-null  object        
 6   original_title         44501 non-null  object        
 7   overview               44501 non-null  object        
 8   popularity             44501 non-null  float64       
 9   production_companies   44501 non-null  object        
 10  production_countries   44501 non-null  object        
 11  release_date           44430 non-null  datetime64[ns]
 12  revenue                44501 non-null  float64       
 13  runtim

Droping duplicates

In [14]:
#Droping duplicates
movies_pd = movies_pd.drop_duplicates()

In [15]:
#df info
movies_pd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44484 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4430 non-null   object        
 1   budget                 44484 non-null  int64         
 2   genres                 44484 non-null  object        
 3   id                     44484 non-null  int64         
 4   imdb_id                44469 non-null  object        
 5   original_language      44123 non-null  object        
 6   original_title         44484 non-null  object        
 7   overview               44484 non-null  object        
 8   popularity             44484 non-null  float64       
 9   production_companies   44484 non-null  object        
 10  production_countries   44484 non-null  object        
 11  release_date           44413 non-null  datetime64[ns]
 12  revenue                44484 non-null  float64       
 13  runtim

Changing 0 into None in numeric data

In [16]:
#Changing 0 into None in numeric data
columns = ['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']
for col in columns:
    movies_pd[col] = movies_pd[col].replace(0, None)

Data such as collection, genres, languages, production companies, countries will be stored as additional nodes in NeoJ4 database. Because of that every of these variables will be saved in additional files. Rest of variables will be saved in one file, that will be used to create movie nodes in NeoJ4.

In [17]:
#Saving cleaned dataframe
columns_to_save = ['id', 'title', 'original_title', 'overview', 'release_date', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']
movies_pd[columns_to_save].to_csv(CLEAN_MOVIES_FILE, index=False)

## TRASNFORMING COLLECTIONS TO ADDITIONAL DATAFRAME

In [18]:
#Creating df for collections
collections_df = movies_pd[['id', 'belongs_to_collection']].dropna()

In [19]:
#Extract collection id and name for each movie
collection_rows = []
for idx, row in collections_df.iterrows():
    movie_id = row['id']
    collection = row['belongs_to_collection']
    if isinstance(collection, str):
        try:
            collection = ast.literal_eval(collection)
        except Exception:
            collection = None
    if isinstance(collection, dict):
        collection_id = collection.get('id')
        collection_name = collection.get('name')
        collection_rows.append({
            'movie_id': movie_id,
            'collection_id': collection_id,
            'collection_name': collection_name
        })
    else:
        print(f"Skipping invalid collection data for movie_id {movie_id}")
collections_df = pd.DataFrame(collection_rows)

In [20]:
#df info
collections_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4430 entries, 0 to 4429
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   movie_id         4430 non-null   int64 
 1   collection_id    4430 non-null   int64 
 2   collection_name  4430 non-null   object
dtypes: int64(2), object(1)
memory usage: 104.0+ KB


In [21]:
#Cleaning Na values in collection_id and empty strings in collection_name
collections_df = collections_df.dropna(subset=['collection_id'])
collections_df = collections_df[collections_df['collection_name'].str.strip().astype(bool)]

In [22]:
#Save collections dataframe to csv
collections_df.to_csv(COLLECTIONS_FILE, index=False)

## TRANSFORMING GENRES INTO ADDITIONAL DATAFRAME

In [23]:
#Creating df fo genres
genres_df = movies_pd[['id', 'genres']].dropna()

In [24]:
#df info
genres_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44484 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      44484 non-null  int64 
 1   genres  44484 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [25]:
#Extract genre id and name for each movie
genre_rows = []
for idx, row in genres_df.iterrows():
    movie_id = row['id']
    genres_list = row['genres']
    if isinstance(genres_list, str):
        try:
            genres_list = ast.literal_eval(genres_list)
        except Exception:
            genres_list = []
    for genre in genres_list:
        if isinstance(genre, dict):
            genre_id = genre.get('id')
            genre_name = genre.get('name')
            genre_rows.append({
                'movie_id': movie_id,
                'genre_id': genre_id,
                'genre_name': genre_name
            })
        else:
            print(f"Skipping invalid genre data for movie_id {movie_id}")
genres_df = pd.DataFrame(genre_rows)

In [26]:
#df info
genres_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89968 entries, 0 to 89967
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   movie_id    89968 non-null  int64 
 1   genre_id    89968 non-null  int64 
 2   genre_name  89968 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.1+ MB


In [27]:
#Clearing Na values in genre_id and empty strings in genre_name
genres_df = genres_df.dropna(subset=['genre_id'])
genres_df = genres_df[genres_df['genre_name'].str.strip().astype(bool)]

In [28]:
#Saving genres dataframe to csv
genres_df.to_csv(GENRE_FILE, index=False)

## TRANSFORFMING PRODUCTION COMPANIES INTO ADDITIONAL DATAFRAME

In [29]:
#Creating df for production companies
prod_comp_df = movies_pd[['id', 'production_companies']].dropna()

In [30]:
#df info
prod_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44484 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    44484 non-null  int64 
 1   production_companies  44484 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [31]:
#Extract production company id and name for each movie
prod_comp_rows = []
for idx, row in prod_comp_df.iterrows():
    movie_id = row['id']
    prod_comp_list = row['production_companies']
    if isinstance(prod_comp_list, str):
        try:
            prod_comp_list = ast.literal_eval(prod_comp_list)
        except Exception:
            prod_comp_list = []
    for prod_comp in prod_comp_list:
        if isinstance(prod_comp, dict):
            prod_comp_id = prod_comp.get('id')
            prod_comp_name = prod_comp.get('name')
            prod_comp_rows.append({
                'movie_id': movie_id,
                'prod_comp_id': prod_comp_id,
                'prod_comp_name': prod_comp_name
            })
        else:
            print(f"Skipping invalid production company data for movie_id {movie_id}")
prod_comp_df = pd.DataFrame(prod_comp_rows)

In [32]:
#df info
prod_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69778 entries, 0 to 69777
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   movie_id        69778 non-null  int64 
 1   prod_comp_id    69778 non-null  int64 
 2   prod_comp_name  69778 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.6+ MB


In [33]:
#Claring Na values in prod_comp_id and empty strings in prod_comp_name
prod_comp_df = prod_comp_df.dropna(subset=['prod_comp_id'])
prod_comp_df = prod_comp_df[prod_comp_df['prod_comp_name'].str.strip().astype(bool)]

In [34]:
#Saving production companies dataframe to csv
prod_comp_df.to_csv(PRODUCTION_COMPANIES_FILE, index=False)

## TRANSFORMING PRODUCTION COUNTRIES INTO ADDITIONAL DATAFRAME

In [35]:
#Creating df for production countries
prod_coun_df = movies_pd[['id', 'production_countries']].dropna()

In [36]:
#df info
prod_coun_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44484 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    44484 non-null  int64 
 1   production_countries  44484 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [37]:
#Extract production country code and name for each movie
prod_coun_rows = []
for idx, row in prod_coun_df.iterrows():
    movie_id = row['id']
    prod_coun_list = row['production_countries']
    if isinstance(prod_coun_list, str):
        try:
            prod_coun_list = ast.literal_eval(prod_coun_list)
        except Exception:
            prod_coun_list = []
    for prod_coun in prod_coun_list:
        if isinstance(prod_coun, dict):
            prod_coun_code = prod_coun.get('iso_3166_1')
            prod_coun_name = iso_3166_dict.get(prod_coun_code, None)
            prod_coun_rows.append({
                'movie_id': movie_id,
                'prod_coun_code': prod_coun_code,
                'prod_coun_name': prod_coun_name
            })
        else:
            print(f"Skipping invalid production country data for movie_id {movie_id}")
prod_coun_df = pd.DataFrame(prod_coun_rows)

In [38]:
#Clearing empty string in prod_coun_code and prod_coun_name
prod_coun_df = prod_coun_df[prod_coun_df['prod_coun_code'].str.strip().astype(bool)]
prod_coun_df = prod_coun_df[prod_coun_df['prod_coun_name'].str.strip().astype(bool)]

In [39]:
#Saving production companies dataframe to csv
prod_coun_df.to_csv(PRODUCTION_COUNTRIES_FILE, index=False)

## TRANSFORMING SPOKEN LANGUAGES INTO ADDITIONAL DATAFRAME

In [40]:
#Creating df for spoken languages
spoken_lang_df = movies_pd[['id', 'spoken_languages']].dropna()

In [41]:
#df info
spoken_lang_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44484 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                44484 non-null  int64 
 1   spoken_languages  44484 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [42]:
#Extract spoken language code for each movie
spoken_lang_rows = []
for idx, row in spoken_lang_df.iterrows():
    movie_id = row['id']
    lang_list = row['spoken_languages']
    if isinstance(lang_list, str):
        try:
            lang_list = ast.literal_eval(lang_list)
        except Exception:
            lang_list = []
        for lang in lang_list:
            if isinstance(lang, dict):
                lang_code = lang.get('iso_639_1')
                lang_name = iso_639_dict.get(lang_code, None)
                if lang_name:
                    spoken_lang_rows.append({
                        'movie_id': movie_id,
                        'lang_code': lang_code,
                        'lang_name': lang_name
                    })
            else:
                print(f"Skipping invalid spoken language data for movie_id {movie_id}")
spoken_lang_df = pd.DataFrame(spoken_lang_rows)

In [43]:
#Clearing empty string in lang_code and lang_name
spoken_lang_df = spoken_lang_df[spoken_lang_df['lang_code'].str.strip().astype(bool)]  
spoken_lang_df = spoken_lang_df[spoken_lang_df['lang_name'].str.strip().astype(bool)]       

In [44]:
#df info
spoken_lang_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51689 entries, 0 to 51688
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie_id   51689 non-null  int64 
 1   lang_code  51689 non-null  object
 2   lang_name  51689 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [45]:
#Saving spoken languages dataframe to csv
spoken_lang_df.to_csv(SPOKEN_LANGUAGES_FILE, index=False)